#  Week 7

##Importing folders

In [ ]:
!pip install -U -q PyDrive


In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)



# After a specific amount of time Pydrive logs out, the enxt function checks for that and if it has happend trys log in again
# you will not to log in to your account just the first time, the next times it will be automatic, we will use this function later on.
def authorizeIfLoggedOut():
  try:
    file_list_GDrive = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()

  except:    
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)


## Importing modules

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:

# this will just check if we are logged out it logs in
authorizeIfLoggedOut()

file_list_GDrive = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()

for file1 in file_list_GDrive:  
    if "EETBS" in file1['title']:
        Folder_ID_EETBS= file1['id']

print(Folder_ID_EETBS)

file_list_EETBS = drive.ListFile({'q': "'%s' in parents and trashed=false" % Folder_ID_EETBS}).GetList()

for file2 in file_list_EETBS:  
    if "Data" in file2['title']:
        Folder_ID_Data= file2['id']

        
file_list_Data = drive.ListFile({'q': "'%s' in parents and trashed=false" % Folder_ID_Data}).GetList()

for file3 in file_list_Data:  
    if "BeamIrradiance.csv" in file3['title']:
        print("BeamIrradiance.csv")
        ID_BeamIrradiance= file3['id']
        file_BeamIrradiance = drive.CreateFile({'id': ID_BeamIrradiance})
        file_BeamIrradiance.GetContentFile('BeamIrradiance.csv')
    if "windows.csv" in file3['title']:
        print("windows.csv exists")
        ID_windows= file3['id']
        file_windows = drive.CreateFile({'id': ID_windows})
        file_windows.GetContentFile('windows.csv') 
    if "IAC_cl.csv" in file3['title']:
        print("IAC_cl.csv exists")
        ID_IAC_cl= file3['id']
        file_IAC_cl = drive.CreateFile({'id': ID_IAC_cl})
        file_IAC_cl.GetContentFile('IAC_cl.csv') 


1caB1_U3vNNcNT6p9OE8ue7GbyOAxs2Oj
windows.csv exists
BeamIrradiance.csv
IAC_cl.csv exists


##DataFrame creation from .csv file

*Creating DataFrame* by simply reading a separate (excel-based) file:

In [ ]:
window_DF = pd.read_csv("windows.csv", sep=";", index_col = 0, header=0) 
window_DF

,Direction,width,Height,Area,Window_ID,Frame_type,Frame_material,U,SHGC,HF,Q heating,SLF,Doh,Xoh,Fshd,IntShading_ID,IntShading_closeness,IAC_cl,IAC,Tx,Ed,ED,PXI,FFs,C_value,CF,Qcooling
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,
east,E,8,1.8,0,5c,Fixed,Wood,0,0,0,0,0,0.9,0.3,0,DrapesLightOpen,0.4,0,0,1.00,0,0,0,0,0,0,0
west,W,8,1.8,0,5c,Fixed,Wood,0,0,0,0,0,0.9,0.3,0,DrapesLightOpen,0.4,0,0,1.00,0,0,0,0,0,0,0
south-Fixed,S,2,1.8,0,5c,Fixed,Wood,0,0,0,0,0,0.9,0.3,0,DrapesLightOpen,0.4,0,0,1.00,0,0,0,0,0,0,0
south-Operable,S,2,1.8,0,5c,Operable,Wood,0,0,0,0,0,0.9,0.3,0,DrapesLightOpen,0.4,0,0,0.64,0,0,0,0,0,0,0


In [ ]:
window_DF.loc[:,"Frame_type"]

Name
east                 Fixed
west                 Fixed
south-Fixed          Fixed
south-Operable    Operable
Name: Frame_type, dtype: object

In [ ]:
window_DF.loc[:,"uselesscolumn"] = "useless"
window_DF

,Direction,width,Height,Area,Window_ID,Frame_type,Frame_material,U,SHGC,HF,Q heating,SLF,Doh,Xoh,Fshd,IntShading_ID,IntShading_closeness,IAC_cl,IAC,Tx,Ed,ED,PXI,FFs,C_value,CF,Qcooling,uselesscolumn
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,
east,E,8,1.8,0,5c,Fixed,Wood,0,0,0,0,0,0.9,0.3,0,DrapesLightOpen,0.4,0,0,1.00,0,0,0,0,0,0,0,useless
west,W,8,1.8,0,5c,Fixed,Wood,0,0,0,0,0,0.9,0.3,0,DrapesLightOpen,0.4,0,0,1.00,0,0,0,0,0,0,0,useless
south-Fixed,S,2,1.8,0,5c,Fixed,Wood,0,0,0,0,0,0.9,0.3,0,DrapesLightOpen,0.4,0,0,1.00,0,0,0,0,0,0,0,useless
south-Operable,S,2,1.8,0,5c,Operable,Wood,0,0,0,0,0,0.9,0.3,0,DrapesLightOpen,0.4,0,0,0.64,0,0,0,0,0,0,0,useless


In [ ]:
window_DF.columns

Index(['Direction', 'width', 'Height', 'Area', 'Window_ID', 'Frame_type',
       'Frame_material', 'U', 'SHGC', 'HF', 'Q heating', 'SLF', 'Doh', 'Xoh',
       'Fshd', 'IntShading_ID', 'IntShading_closeness', 'IAC_cl', 'IAC', 'Tx',
       'Ed', 'ED', 'PXI', 'FFs', 'C_value', 'CF', 'Qcooling', 'uselesscolumn'],
      dtype='object')

Index column is not a column:

In [ ]:
window_DF.index

Index(['east', 'west', 'south-Fixed', 'south-Operable'], dtype='object', name='Name')

Updating the values of "Area" column based on the relevant width and height:

In [ ]:
window_DF.loc[:,"Area"] = window_DF.loc[:,"width"]*window_DF.loc[:,"Height"]

In [ ]:
window_DF

,Direction,width,Height,Area,Window_ID,Frame_type,Frame_material,U,SHGC,HF,Q heating,SLF,Doh,Xoh,Fshd,IntShading_ID,IntShading_closeness,IAC_cl,IAC,Tx,Ed,ED,PXI,FFs,C_value,CF,Qcooling,uselesscolumn
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,
east,E,8,1.8,14.4,5c,Fixed,Wood,0,0,0,0,0,0.9,0.3,0,DrapesLightOpen,0.4,0,0,1.00,0,0,0,0,0,0,0,useless
west,W,8,1.8,14.4,5c,Fixed,Wood,0,0,0,0,0,0.9,0.3,0,DrapesLightOpen,0.4,0,0,1.00,0,0,0,0,0,0,0,useless
south-Fixed,S,2,1.8,3.6,5c,Fixed,Wood,0,0,0,0,0,0.9,0.3,0,DrapesLightOpen,0.4,0,0,1.00,0,0,0,0,0,0,0,useless
south-Operable,S,2,1.8,3.6,5c,Operable,Wood,0,0,0,0,0,0.9,0.3,0,DrapesLightOpen,0.4,0,0,0.64,0,0,0,0,0,0,0,useless


Saving the file (in different formats):

In [ ]:
name_modified_windows = "window_modified.csv"
window_DF.to_csv(name_modified_windows,sep=";")

In [ ]:
window_DF.to_excel("window_modified.xlsx")

In [ ]:
window_DF.to_html("window_modified.html")

In [ ]:
IAC_cl_DF = pd.read_csv("IAC_cl.csv",sep=";",index_col=1,header=0)
IAC_cl_DF

,Type,DrapesLightOpen,DrapesDarkClosed,DrapesLightClosed,RollerOpaqueDark,RollerOpqaueWhite,RollerTranslucentLight,BlindsMedium,BlindsDark
ID,,,,,,,,,
1a,clear_1Layer,0.64,0.71,0.45,0.64,0.34,0.44,0.74,0.66
1c,HeatAbsorbing_1Layer,0.68,0.72,0.50,0.67,0.40,0.49,0.76,0.69
5a,Clear_2Layer,0.72,0.81,0.57,0.76,0.48,0.55,0.82,0.74
17c,lowEpsillonHighSolar_2Layers,0.76,0.86,0.64,0.82,0.57,0.62,0.86,0.79
25a,lowEpsillonLowSolar_2Layers,0.79,0.88,0.68,0.85,0.60,0.66,0.88,0.82
5c,HeatAbsorbing_2Layers,0.73,0.82,0.59,0.77,0.51,0.58,0.83,0.76


In [ ]:
IAC_cl_DF.columns

Index(['Type', 'DrapesLightOpen', 'DrapesDarkClosed', 'DrapesLightClosed',
       'RollerOpaqueDark', 'RollerOpqaueWhite', 'RollerTranslucentLight',
       'BlindsMedium', 'BlindsDark'],
      dtype='object')

In [ ]:
IAC_cl_DF.index

Index(['1a', '1c', '5a', '17c', '25a', '5c'], dtype='object', name='ID')

##Some useful commands for large datasets

In [ ]:
IAC_cl_DF.head(3)

,Type,DrapesLightOpen,DrapesDarkClosed,DrapesLightClosed,RollerOpaqueDark,RollerOpqaueWhite,RollerTranslucentLight,BlindsMedium,BlindsDark
ID,,,,,,,,,
1a,clear_1Layer,0.64,0.71,0.45,0.64,0.34,0.44,0.74,0.66
1c,HeatAbsorbing_1Layer,0.68,0.72,0.50,0.67,0.40,0.49,0.76,0.69
5a,Clear_2Layer,0.72,0.81,0.57,0.76,0.48,0.55,0.82,0.74


In [ ]:
IAC_cl_DF.tail(3)

,Type,DrapesLightOpen,DrapesDarkClosed,DrapesLightClosed,RollerOpaqueDark,RollerOpqaueWhite,RollerTranslucentLight,BlindsMedium,BlindsDark
ID,,,,,,,,,
17c,lowEpsillonHighSolar_2Layers,0.76,0.86,0.64,0.82,0.57,0.62,0.86,0.79
25a,lowEpsillonLowSolar_2Layers,0.79,0.88,0.68,0.85,0.60,0.66,0.88,0.82
5c,HeatAbsorbing_2Layers,0.73,0.82,0.59,0.77,0.51,0.58,0.83,0.76


In [ ]:
windowID = "5c"
intShadingID = "DrapesLightOpen"
value_IAC_cl = IAC_cl_DF.loc[windowID,intShadingID]
value_IAC_cl

0.73

Create a single function that for shading type (fixing the shading type to DrapesLightOpen) reads the table and gives you the value being provided with the type of the window:

In [ ]:
def IAC_cl_finder_DrapesLightOpen(windowID):
  intShadingID = "DrapesLightOpen"
  IAC_cl_DF = pd.read_csv("IAC_cl.csv",sep=";",index_col=1,header=0)
  value_IAC_cl = IAC_cl_DF.loc[windowID,intShadingID]
  return value_IAC_cl

In [ ]:
IAC_cl_finder_DrapesLightOpen("5c")

0.73

This function first, read the file that we want to extract data from, and then extract the data based on the defined procedure.

In [ ]:
window_ID_column = window_DF.loc[:,"Window_ID"]
window_ID_column.apply(IAC_cl_finder_DrapesLightOpen)

Name
east              0.73
west              0.73
south-Fixed       0.73
south-Operable    0.73
Name: Window_ID, dtype: float64

We can also implement several commands continuously.

In [ ]:
window_DF.loc[:, "IAC_cl"] = window_DF.loc[:,"Window_ID"].apply(IAC_cl_finder_DrapesLightOpen)
window_DF

,Direction,width,Height,Area,Window_ID,Frame_type,Frame_material,U,SHGC,HF,Q heating,SLF,Doh,Xoh,Fshd,IntShading_ID,IntShading_closeness,IAC_cl,IAC,Tx,Ed,ED,PXI,FFs,C_value,CF,Qcooling,uselesscolumn
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,
east,E,8,1.8,14.4,5c,Fixed,Wood,0,0,0,0,0,0.9,0.3,0,DrapesLightOpen,0.4,0.73,0,1.00,0,0,0,0,0,0,0,useless
west,W,8,1.8,14.4,5c,Fixed,Wood,0,0,0,0,0,0.9,0.3,0,DrapesLightOpen,0.4,0.73,0,1.00,0,0,0,0,0,0,0,useless
south-Fixed,S,2,1.8,3.6,5c,Fixed,Wood,0,0,0,0,0,0.9,0.3,0,DrapesLightOpen,0.4,0.73,0,1.00,0,0,0,0,0,0,0,useless
south-Operable,S,2,1.8,3.6,5c,Operable,Wood,0,0,0,0,0,0.9,0.3,0,DrapesLightOpen,0.4,0.73,0,0.64,0,0,0,0,0,0,0,useless


We did a wrong assumption; our function was ot receiving the intShadingID as an input. Let's correct it:

In [ ]:
def IAC_cl_finder_V2(windowID, indShadingID):
  IAC_cl_DF = pd.read_csv("IAC_cl.csv",sep=";",index_col=1,header=0)
  value_IAC_cl = IAC_cl_DF.loc[windowID,intShadingID]
  return value_IAC_cl

So, we now have a **function with two inputs**. This is very common in our cases.

In [ ]:
IAC_cl_finder_V2("5c","RollerOpaqueDark")

0.73

We can put our multiple input, in a single input:

In [ ]:
def IAC_cl_finder(row):
  window_IC = row["Window_ID"]
  intShading_ID = row["IntShading_ID"]
  IAC_cl_DF = pd.read_csv("IAC_cl.csv",sep=";",index_col=1,header=0)
  value_IAC_cl = IAC_cl_DF.loc[windowID,intShadingID]
  return value_IAC_cl

In [ ]:
window_DF.apply(IAC_cl_finder,axis=1)

Name
east              0.73
west              0.73
south-Fixed       0.73
south-Operable    0.73
dtype: float64

In fact, all we have done recently is a better way to extract the data from our DataFram which is necessary in case of having large DataFrame.

In [ ]:
window_DF.loc[:,"IAC_cl"] = window_DF.loc[:,"Window_ID"].apply(IAC_cl_finder_DrapesLightOpen)
window_DF

,Direction,width,Height,Area,Window_ID,Frame_type,Frame_material,U,SHGC,HF,Q heating,SLF,Doh,Xoh,Fshd,IntShading_ID,IntShading_closeness,IAC_cl,IAC,Tx,Ed,ED,PXI,FFs,C_value,CF,Qcooling,uselesscolumn
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,
east,E,8,1.8,14.4,5c,Fixed,Wood,0,0,0,0,0,0.9,0.3,0,DrapesLightOpen,0.4,0.73,0,1.00,0,0,0,0,0,0,0,useless
west,W,8,1.8,14.4,5c,Fixed,Wood,0,0,0,0,0,0.9,0.3,0,DrapesLightOpen,0.4,0.73,0,1.00,0,0,0,0,0,0,0,useless
south-Fixed,S,2,1.8,3.6,5c,Fixed,Wood,0,0,0,0,0,0.9,0.3,0,DrapesLightOpen,0.4,0.73,0,1.00,0,0,0,0,0,0,0,useless
south-Operable,S,2,1.8,3.6,5c,Operable,Wood,0,0,0,0,0,0.9,0.3,0,DrapesLightOpen,0.4,0.73,0,0.64,0,0,0,0,0,0,0,useless


IAC = 1+(IAC_cl-1)*closeness

In [ ]:
1+(window_DF.loc[:,"IAC_cl"]-1)*window_DF.loc[:,"IntShading_closeness"]

Name
east              0.892
west              0.892
south-Fixed       0.892
south-Operable    0.892
dtype: float64

In [ ]:
window_DF.loc[:,"IAC"] = 1+(window_DF.loc[:,"IAC_cl"]-1)*window_DF.loc[:,"IntShading_closeness"]

How to see only some necessary columns:

In [ ]:
window_DF.loc[:,["IntShading_closeness","IAC_cl","IAC"]]

,IntShading_closeness,IAC_cl,IAC
Name,,,
east,0.4,0.73,0.892
west,0.4,0.73,0.892
south-Fixed,0.4,0.73,0.892
south-Operable,0.4,0.73,0.892


##Assignment 7

Use the same procedure and write function that read the diffuse radiation, the direct radiation, the SLF, and F_shd and write the PXI back on the window_DF and update the columns.
* Note: import the relevant file (if using your file!) 